## Importing all dependencies 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import re
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

## Loading of Data

In [2]:
df = pd.read_csv(r'C:\Users\Edna\Documents\GitHub\Haile_Resort_Sentiment_Analysis\data\processed\Haile_and_kaggledata.csv')
df['Sentiment'].value_counts()

Sentiment
Positive    9191
Negative    1240
Neutral      187
Name: count, dtype: int64

## Data Cleaning and Preprocessing

In [3]:
#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)
print(stop_words)

{'before', 'my', "that'll", 'through', 'very', 'of', 'down', "aren't", 'i', 'how', 'wouldn', 'out', 'all', 'no', 'm', 'above', 'there', 'too', 'he', 'have', 'each', 'being', 'against', "isn't", 'while', 'couldn', 'isn', 'needn', 'or', "couldn't", 'himself', 'hers', 'by', 'hadn', "shouldn't", "shan't", 'our', 'theirs', 'with', 'in', "doesn't", 'don', 'could', 'does', 'own', 'yours', 'which', 'between', 'd', 'from', "you're", 'so', 'most', 're', 'were', 'here', 'more', 'other', 'her', 'because', 'into', 'can', 't', 'you', 'as', 'whom', 'am', 'ours', 'was', 'only', 'and', 'but', 'would', 'if', 'off', "you've", 'his', 'again', 'themselves', 'haven', "needn't", 'shall', 'some', 'myself', 'been', "she's", "wouldn't", 'this', 'wasn', 'mightn', "you'd", 'over', 'what', 'didn', "weren't", 'might', 'at', 'when', 'y', 'such', 'ma', 'until', 'won', "mustn't", 'yourselves', 'further', 'are', "should've", 'herself', 'has', 'both', 'now', 'a', "don't", 'that', 'she', 'where', "hasn't", 'than', 'these

In [4]:
'''-----------------------------Data Cleaning and Preprocessing pipeline----------------------------------'''

#Removing special character
def remove_special_character(content):
    return re.sub('\W+',' ', content )#re.sub('\[[^&@#!]]*\]', '', content)

# Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the stopwords from text
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expansion of english contractions
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)
    return content

#Data preprocessing
def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    
    content = remove_stopwords(content)    
    return content

In [5]:
#load cleaned data
pd.options.display.max_colwidth = 1000
#Data cleaning
df['Reviews_clean']=df['reviews.text'].apply(data_cleaning)
df[['Reviews_clean','reviews.text']].head(5)

,Reviews_clean,reviews.text
0,courteous friendly service nice spa facilities delicious breakfast,very courteous and friendly service nice spa facilities delicious breakfast
1,stay haile grand addis hotel nothing short exceptional opulent decor impeccable service every aspect exuded luxury rooms offered great views including great views addis ababa night complemented indulgent amenities truly enhanced experience anyone seeking blend comfort sophistication haile grand addis hotel perfect choice offering unparalleled value luxurious stay,my stay at the haile grand addis hotel was nothing short of exceptional from the opulent decor to the impeccable service every aspect exuded luxury the rooms offered great views including great views of addis ababa at night complemented by indulgent amenities that truly enhanced the experience for anyone seeking a blend of comfort and sophistication the haile grand addis hotel is the perfect choice offering unparalleled value for a luxurious stay
2,stay haile grand stay experience staff friendly helped us book taxis traditional dance group evening rooms amazing definitely enjoyed stay highly recommend haile grand,our stay at haile grand was more than a stay it was an experience the staff were so friendly they helped us book taxis there was a traditional dance group for the evening the rooms are amazing and we definitely enjoyed our stay i would highly recommend haile grand
3,beautiful modern still charming old building spacious bright feels luxurious safe outside hotel feels bit unsafe luxury building opposite reality regular ethiopian go area feel unsafe however ethopian people extremely friendly welcoming,beautiful very modern and still charming of old building very spacious in bright feels luxurious and safe once you are outside the hotel feels bit unsafe this luxury is building is the opposite reality what regular ethiopian go through so area can feel unsafe however ethopian people are extremely friendly and very welcoming
4,hotel beautiful decoration layout rooms lighting spacious extremely clean feel secure steps going spa challenging tired however keep fit need go gym,the hotel is beautiful from decoration to layout of the rooms lighting and very spacious extremely clean you feel very secure the steps going up to spa are can be challenging if you are tired however it can keep you fit don t need to go to the gym


# Feature Engineering

In [7]:
# Mapping sentiment to numerical labels
label_mapping = {
    'Positive': 1,
    'Negative': 0,
    'Neutral': 2
}
df['Label'] = df['Sentiment'].map(label_mapping)

df[['Sentiment','Label']].tail(10)


,Sentiment,Label
10608,Positive,1
10609,Neutral,2
10610,Positive,1
10611,Positive,1
10612,Positive,1
10613,Positive,1
10614,Positive,1
10615,Positive,1
10616,Positive,1
10617,Positive,1


In [8]:
#Importing dependencies for feature engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from prettytable import PrettyTable
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

## Lemmatization

In [9]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

## Vectoization with Count Vectorizer and TDIDF Vectorizer with Unigram

In [13]:
train,test=train_test_split(df,test_size=.2,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']


## Feature Importance with Logistic Regression and Count Vectorizer with unigram

In [15]:
# Initialize and train the logistic regression model
lgr = LogisticRegression()
lgr.fit(x_train_count, y_train)  # Fitting the model with training data

# Create a PrettyTable instance for feature importance
importantfeature = PrettyTable(["Feature", "Score"])

# Limit to the first 200 features or fewer
for i, (feature, importance) in enumerate(zip(countvect.get_feature_names_out(), lgr.coef_[0])):
    if i < 200:
        importantfeature.add_row([feature, importance])
    else:
        break

print(importantfeature)

+---------------+------------------------+
|    Feature    |         Score          |
+---------------+------------------------+
|      able     | -0.013112063955995316  |
|   absolutely  |  0.24487297516711223   |
|     access    |  0.18464689977395324   |
| accommodating |  -0.33474856624080085  |
| accommodation |  -0.2155076936656998   |
|     across    |  -0.1918297405252031   |
|    actually   |   0.3027296683844447   |
|      air      | -0.048308718385809385  |
|    airport    |  -0.20809609199816617  |
|     almost    |    0.3823631130945     |
|     along     |  -0.16439981167785223  |
|      also     |  0.23280603728917337   |
|    although   |   0.1773466602105364   |
|     always    |  -0.44425553013807456  |
|    amazing    |  -0.23773840541421518  |
|    amenity    | -0.061464131891424134  |
|    another    |  -0.15679433863994952  |
|     anyone    |   0.3718933857530027   |
|    anything   |  -0.31138896595276044  |
|   apologize   |   0.8412859291641744   |
|   appreci

## Feature Importance with TFIDF vectorizer and Logistic Regression with Unigram 

In [16]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------+-----------------------+
|    Feature    |         Score         |
+---------------+-----------------------+
|      able     |  -0.24048506578150053 |
|   absolutely  |  0.26190318805790913  |
|     access    |   0.121346380086558   |
| accommodating |  -0.8486288356908819  |
| accommodation |  -0.1901173510510828  |
|     across    |  -0.17947602742753482 |
|    actually   |  0.30050119639605755  |
|      air      |  0.28476148004411933  |
|    airport    |  -0.6177436840860248  |
|     almost    |  0.49775380775139105  |
|     along     |  -0.05598472638343983 |
|      also     |   0.3326528606104226  |
|    although   |  -0.13808688414979187 |
|     always    |   -1.081034645953989  |
|    amazing    |  -1.2594741739970963  |
|    amenity    | -0.038382227679508975 |
|    another    |   0.6188543162868961  |
|     anyone    |   0.7996639163696739  |
|    anything   |  -0.2134005250095025  |
|   apologize   |   2.238772659675674   |
|   appreciate  |  0.0067290763083

## Vectorization with Count Vectorizer and TDIDF Vectorizer with Bigram

In [17]:
train,test=train_test_split(df,test_size=.2,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer with Bigram

In [18]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-------------------------+------------------------+
|         Feature         |         Score          |
+-------------------------+------------------------+
|         able get        |  -0.21890052726349385  |
|      across street      |  -0.07753789383457413  |
|     air conditioning    |   0.2289412801950454   |
|       although not      |  0.08245552533985029   |
|      another chance     |  0.08971560536881988   |
|   appreciate feedback   |  0.08433764693500312   |
|     appreciate kind     |   0.6444271327054094   |
|    appreciate taking    |   0.3261166969127025   |
|        area hotel       |   0.4659536917704698   |
|         area not        |  -0.2580353768636342   |
|      around corner      |  0.06846298631484746   |
|        away home        |   0.1527254959344189   |
|        back hotel       |  0.05172478063516502   |
|        back next        |  -0.40805086218913617  |
|        back room        |   -0.263646171409009   |
|        back soon        |  -0.70530865684306

## Feature Importance with Logistic Regression and TFIDF Vectorizer with Bigram

In [19]:
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=50:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------+-----------------------+
|       Feature       |         Score         |
+---------------------+-----------------------+
|       able get      |  -0.06179575907214881 |
|    across street    |  0.05346607967433101  |
|   air conditioning  |  0.40951624466838227  |
|     although not    |  -0.11210409445707885 |
|    another chance   |   0.5235209854253698  |
| appreciate feedback |  0.11659358295240888  |
|   appreciate kind   |   0.1939505883560886  |
|  appreciate taking  |  0.40867430678029626  |
|      area hotel     |  0.27988203353388497  |
|       area not      |  -0.41042319516403314 |
|    around corner    |  -0.1895043575447604  |
|      away home      |  -0.09422517666657233 |
|      back hotel     |   0.5291648776581196  |
|      back next      |  -0.45823942433811027 |
|      back room      |   0.1002317236003758  |
|      back soon      |  -0.8781177933662826  |
|    bar restaurant   |  -0.6779365332019973  |
|   beautiful hotel   |  -0.272889691430

## Vectorization with Count Vectorizer and TFIDF Vectorizer with Trigram

In [20]:
train,test=train_test_split(df,test_size=.2,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer with Trigram

In [21]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------------+-----------------------+
|             Feature             |         Score         |
+---------------------------------+-----------------------+
|       able take advantage       |  -0.21725516928802396 |
|      accept sincere apology     |   0.670188424634068   |
|       across street space       |  -0.2687529109144093  |
|    alex blancogeneral manager   |  -0.42584382542014887 |
|  amber bertschguest experience  |  0.06899433198812584  |
|      another chance provide     |  0.033700649323499574 |
|     appreciate feedback hope    |  -0.8831001061099406  |
|       appreciate kind word      |  -0.29011951267711344 |
|      appreciate taking time     |   0.5190256224861498  |
|          back next time         |  -0.47756044718084806 |
|          back next trip         |  -0.32822047901839985 |
|            back see u           |  -0.13196864349491563 |
|       back soon sincerely       |   0.5521545617725613  |
|           back stay u           |  -0.

## Feature Importance with Logistic Regression and TFIDF Vectorizer with Trigram

In [22]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------------+-----------------------+
|             Feature             |         Score         |
+---------------------------------+-----------------------+
|       able take advantage       |  -0.33204293824770936 |
|      accept sincere apology     |   0.9985961294172977  |
|       across street space       |  -0.32335897260013535 |
|    alex blancogeneral manager   |  -0.29621991520120877 |
|  amber bertschguest experience  |  -0.12481067915431207 |
|      another chance provide     |   0.2549773057414858  |
|     appreciate feedback hope    |  -0.6749491592847403  |
|       appreciate kind word      |  -0.43229513690473353 |
|      appreciate taking time     |   0.5577668670769297  |
|          back next time         |  -0.5977627534395106  |
|          back next trip         |  -0.35016185436272196 |
|            back see u           |  -0.3832621048977683  |
|       back soon sincerely       |  0.37562005559813905  |
|           back stay u           |  -0.

## Feature Selection with Chi squared

In [23]:
from sklearn.feature_selection import chi2
import numpy as np

# Fit the TF-IDF vectorizer
tfidfvect = TfidfVectorizer(ngram_range=(1, 3), min_df=1)
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean'])

N = 10
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names_out())[indices]

    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]

    print(f"\nCategory: {category}")
    print(f"Unigrams: {unigrams[-N:]}")
    print(f"Bigrams: {bigrams[-N:]}")
    print(f"Trigrams: {trigrams[-N:]}")



Category: 1
Unigrams: ['told', 'disgusting', 'horrible', 'exceptional', 'moremore', 'not', 'bad', 'passable', 'dirty', 'poor']
Bigrams: ['room smelled', 'room not', 'not cleaned', 'not recommend', 'smelled like', 'not even', 'not stay', 'not clean', 'never stay', 'not work']
Trigrams: ['staff horrible us', 'give us another', 'dislike front desk', 'apologize not exceeding', 'poor customer service', 'us another chance', 'hope give us', 'called front desk', 'room not clean', 'not recommend hotel']

Category: 0
Unigrams: ['terrible', 'exceptional', 'told', 'horrible', 'disgusting', 'not', 'bad', 'dirty', 'passable', 'poor']
Bigrams: ['room not', 'room smelled', 'us another', 'not recommend', 'smelled like', 'not stay', 'not even', 'not clean', 'not work', 'never stay']
Trigrams: ['like apologize not', 'dislike front desk', 'give us another', 'apologize not exceeding', 'poor customer service', 'us another chance', 'called front desk', 'hope give us', 'room not clean', 'not recommend hotel'